# Hugging Face NER

[NER Tutorial](https://huggingface.co/learn/llm-course/en/chapter7/2)

Uses the [CoNLL-2003 dataset](https://huggingface.co/datasets/conll2003), which has news stories from Reuters.


In [26]:
# !pip install datasets transformers
!pip install torch

^C


In [5]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003", trust_remote_code=True)

Generating test split: 100%|██████████| 3453/3453 [00:00<00:00, 4592.02 examples/s]


In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [10]:
# look at single text
sample = raw_datasets["train"][0]
for k, v in sample.items():
    print(k, ": ", v)

id :  0
tokens :  ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
pos_tags :  [22, 42, 16, 21, 35, 37, 16, 21, 7]
chunk_tags :  [11, 21, 11, 12, 21, 22, 11, 12, 0]
ner_tags :  [3, 0, 7, 0, 0, 0, 7, 0, 0]


In [11]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature


Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [ ]:
# O means the word doesn’t correspond to any entity.
# B-PER/I-PER means the word corresponds to the beginning of/is inside a person entity.
# B-ORG/I-ORG means the word corresponds to the beginning of/is inside an organization entity.
# B-LOC/I-LOC means the word corresponds to the beginning of/is inside a location entity.
# B-MISC/I-MISC means the word corresponds to the beginning of/is inside a miscellaneous entity.

label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [15]:
len(raw_datasets["train"])

14041

7046

In [24]:
import random
idx = random.choice(range(len(raw_datasets["train"])))
words = raw_datasets["train"][idx]["tokens"]
labels = raw_datasets["train"][idx]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

* Securities and   Exchange Commission acts to improve stock-trade prices for investors . 
O B-ORG      I-ORG I-ORG    I-ORG      O    O  O       O           O      O   O         O 


In [25]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
